In [ ]:
import plotly.express as px
import numpy as np
import duckdb
from tqdm import tqdm
import plotly

In [ ]:
import sys
sys.path.append("/home/ubuntu/sky_workdir/encoding-schemes")

from encoding_schemes import get_deterministic_adherence_fn

In [ ]:
import ray

ray.init()

In [ ]:
import os
import psycopg2
import json

conn_string = os.environ["SUPABASE_CONNECTION_URL"]

conn = psycopg2.connect(conn_string)

import pandas as pd

In [ ]:
sel_str = """
-- NuminaMath CoT Rerun
 (
     (data->'experiment_tags'->'numina_math_cot_rerun')::BOOL
     AND (NOT (data->'force_overwrite')::BOOL OR data->'force_overwrite' IS NULL)
     AND (
         (data->'experiment_params'->'sampling_params'->'n')::INT = 4
         OR ((data->'experiment_params'->'model')::TEXT LIKE '%gpt%' AND (data->'experiment_params'->'sft_params'->'batch_size')::INT != 48)
     )
  )
"""

df = pd.read_sql(f"""
SELECT * FROM public.encoding_schemes 
    WHERE 

{sel_str}


ORDER BY created_at DESC
""", conn)

df.head()

In [ ]:
root_dir = "/home/ubuntu/sky_workdir/encoding-schemes/output"

In [ ]:
l_examples = df.to_dict('records')

# Code

In [ ]:

def bootstrap_ci(data, statistic=np.mean, alpha=0.05, n_boot=10_000, random_state=None):
    """
    Returns (point_estimate, low_CI, high_CI) for given 1D data.
    Works with bool, int, or float data.
    """
    x = np.asarray(data).astype(float)  # ensure numeric
    x = x[~np.isnan(x)]
    if len(x) == 0:
        raise ValueError("No valid data for bootstrapping.")

    rng = np.random.default_rng(random_state)
    n = len(x)

    # Draw bootstrap samples
    idx = rng.integers(0, n, size=(n_boot, n))
    samples = x[idx]

    # Apply statistic row-wise
    stats = np.apply_along_axis(statistic, 1, samples)

    point = statistic(x)
    lo = np.percentile(stats, 100 * (alpha / 2))
    hi = np.percentile(stats, 100 * (1 - alpha / 2))
    return point, lo, hi

In [ ]:
import tiktoken

encoding = tiktoken.get_encoding("cl100k_base")


@ray.remote
def count_tokens_from_messages(s):
    try:
        return len(encoding.encode(s, disallowed_special=()))
    except ValueError as e:
        print(e)
        return 0


In [ ]:
@ray.remote(num_cpus=2, memory=32 * 1024 * 1024 * 1024)
def compute_translation_token_count(example, df_data):
    sys.path.append("/home/ubuntu/sky_workdir/encoding-schemes")

    from orchestration.experiment_meta_saver import compute_experiment_hash
    from utils.io_utils import read_large_parquet

    from transformers import AutoTokenizer

    model = example["data"]["experiment_params"]["model"]
    if "gpt" in model or "claude" in model:
        print(f"Overriding tokenizer for {model} with gpt-oss 120b tokenizer because it was detected as a GPT/Claude model!")
        model = "openai/gpt-oss-120b"

    tokenizer = AutoTokenizer.from_pretrained(model)

    return df_data['reference_solution'].map(lambda x: len(tokenizer.encode(x)))


In [ ]:
from tqdm import tqdm

In [ ]:
def compute_ci_cols(example, df, col_name, transformation_fn):
    s_transformed = transformation_fn(df[col_name])
    if np.isscalar(s_transformed) and np.isnan(s_transformed):
        print(f"Warning: {col_name} was all NaN, ignoring!")
        return
    mid, lo, hi = bootstrap_ci(s_transformed)
    example[col_name] = mid
    example[f'{col_name}_low_ci'] = mid - lo
    example[f'{col_name}_hi_ci'] = hi - mid

In [ ]:
def compute_multi_row_transformation(df, row_transform_fn, col_name, agg_fn):
    df[col_name] = df.apply(row_transform_fn, axis=1)

In [ ]:

def _score_rollouts(rollouts):
    # rollouts: list/array of rollout sequences; may also be None/np.nan/scalar
    if rollouts is None or (isinstance(rollouts, float) and np.isnan(rollouts)):
        return np.nan

    vals = []
    for r in rollouts:
        # skip None/NaN
        if r is None or (isinstance(r, float) and np.isnan(r)):
            continue
        vals.append(-np.nansum(r))
    return np.nanmean(vals) if len(vals) else np.nan


def patch_gpt_api_log_loss(example):
    experiment_hash = example['experiment_hash']

    translation_loss = os.path.join(root_dir, experiment_hash, "data", f"validation_reverse_translation_math500_meta.json")
    with open(translation_loss, "r") as fp:
        example["backtranslation_gt_logprobs"] = translation_loss["valid_loss"]

    # need to be validation loss on 512k...
    validation_loss = os.path.join(root_dir, experiment_hash, "data", f"validation_reverse_translation_math500_meta.json")
    with open(validation_loss, "r") as fp:
        example["backtranslation_gt_logprobs"] = translation_loss["valid_loss"]


@ray.remote
def process_single_example(example):
    experiment_hash = example['experiment_hash']
    
    target_path = os.path.join(root_dir, example['experiment_hash'], "data", "joined_output.parquet")
    if not os.path.exists(target_path):
        print(f"!!!!!! {target_path} missing !!!!!!!")
        return example
    
    df_data = pd.read_parquet(target_path)

    d_col_to_transform = {
        'cot_gt_logprobs' : lambda s: np.nansum(s.map(_score_rollouts)),
        'generated_cot_is_correct' : np.mean,  # was np.mean
        'backtranslation_gt_logprobs' : lambda s: np.nanmean(s.map(_score_rollouts)),
        'backtranslation_bleu_scores' : np.mean,  # was np.mean
        'generated_cot_adhered_encoding_style': np.mean  # was np.mean
    }
    for col, fn in d_col_to_transform.items():
        if col not in df_data:
            print(col)
            print(df_data.head())
            print(example)
            raise Exception(str(col) + "\n" + str(df_data.head()) + "\n" + str(example))

        compute_ci_cols(example, df_data, col, fn)

    df_data["num_tokens_translation_output"] = ray.get(compute_translation_token_count.remote(example, df_data))

    d_and_cols = {
        'adherent_and_correct': (
            lambda r: np.nanmean( \
                np.array(r['generated_cot_is_correct']).astype(bool) & \
                np.array(r['generated_cot_adhered_encoding_style']).astype(bool) \
            ),
            np.nanmean
        ),
        'total_translation_loss': (
            lambda r: np.nanmean( \
                np.array(r['num_tokens_translation_output']) * \
                np.array(np.nanmean(_score_rollouts(r['backtranslation_gt_logprobs'])), dtype=np.float64) \
            ),
            np.nanmean
        ),
    }
    for col, (transform_fn, agg_fn) in d_and_cols.items():
        compute_multi_row_transformation(df_data, transform_fn, col, agg_fn)
        if df_data[col].isna().sum() != len(df_data):
            compute_ci_cols(example, df_data, col, lambda x: x)

    if "gpt" in example["data"]["experiment_params"]["model"] and "use_api_sft_model_for_sampling" in example["data"]["experiment_params"]:
        with open(os.path.join(root_dir, example['experiment_hash'], "data", f"validation_reverse_translation_math500_meta.json"), "r") as fp:
            d_model_meta = json.load(fp)

        example["backtranslation_gt_logprobs"] = d_model_meta["valid_loss"]
        example["total_translation_loss"] = d_model_meta["valid_loss"] * np.nanmean(df_data["num_tokens_translation_output"])
        example["total_translation_loss_low_ci"] = 0.0
        example["total_translation_loss_hi_ci"] = 0.0

    pretraining_prevalence_path = os.path.join('/home/ubuntu/sky_workdir/encoding-schemes', 'output', experiment_hash, 'data', 'num_pretraining_4grams_redpajama.json')
    if os.path.exists(pretraining_prevalence_path):
        with open(pretraining_prevalence_path, "r") as fp:
            d_pretraining_prevalence = json.load(fp)

        example["pretraining_prevalence"] = d_pretraining_prevalence["num_occurrences"]

    for col in df_data.columns:
        example[f"{col}_df"] = df_data[col]

    return example


l_new_examples = [None for _ in range(len(l_examples))]

for i, example in tqdm(enumerate(l_examples)):
    # l_examples[i] = process_single_example(example)
    l_new_examples[i] = process_single_example.remote(example)

for i, example in tqdm(enumerate(l_new_examples)):
    try:
        l_new_examples[i] = ray.get(example)
    except ray.exceptions.RayTaskError as e:
        l_new_examples[i] = l_examples[i]
        print(e)

l_examples = l_new_examples

In [ ]:
def humanize_number(num: float) -> str:
    """
    Converts a number into a human-readable string with k, M, or B suffixes.
    
    Args:
        num (float): The number to format.
    
    Returns:
        str: Human-readable string representation.
    """
    if num >= 1_000_000_000:
        return f"{num / 1_000_000_000:.1f}B"
    elif num >= 1_000_000:
        return f"{num / 1_000_000:.1f}M"
    elif num >= 1_000:
        return f"{num / 1_000:.1f}k"
    else:
        return str(num)

In [ ]:
import re

def parse_params(model):
    if 'gpt' in model:
        if 'nano' in model:
            return 0
        elif 'mini' in model:
            return 1
        else:
            return 2


    if 'claude' in model:
        if 'haiku' in model:
            return 3
        elif 'sonnet' in model:
            return 4
        else:
            return 5
    
    return int(re.search("([0-9]+)B", model).group(1))

In [ ]:
df_viz = pd.DataFrame(l_examples)

orig_len = len(df_viz)

# df_viz = df_viz[df_viz['cot_gt_logprobs'].notna()]

new_len = len(df_viz)
if orig_len != new_len:
    print(f"Lost {orig_len - new_len} examples from na logprobs")

df_viz['encoding_scheme'] = df_viz['data'].map(lambda x: x['experiment_params']['encoding_scheme'])
df_viz['model'] = df_viz['data'].map(lambda x: x['experiment_params']['model'])

try:
    df_viz['model_size'] = df_viz['model'].map(parse_params)
except Exception as e:
    print(e)
df_viz['input_type'] = df_viz['data'].map(lambda x: "_".join(x['experiment_name'].split("_")[:2]))

df_viz['n_few_shot_examples'] = df_viz['data'].map(lambda x: x['experiment_params'].get('n_few_shot_examples', None))

df_viz['Adherence Calculation Method'] = df_viz['encoding_scheme'].map(lambda x: get_deterministic_adherence_fn(x, None) is not None).map({ True: 'deterministic', False: 'Sonnet 4 judge'})

try:
    df_viz['total_train_tok'] = df_viz['n_total_train_tok'].map(humanize_number)
except Exception as e:
    print(e)


In [ ]:
filter_set = ['math_cot']

In [ ]:
df_viz_tmp = df_viz[df_viz['input_type'].isin(filter_set)]
df_viz_tmp = df_viz_tmp[df_viz_tmp['model'] != 'Qwen/Qwen2.5-7B']

df_viz_tmp['model'] = df_viz_tmp['model'].str.split('-2025').str[0]
df_viz_tmp['model'] = df_viz_tmp['model'].str.split('-latest').str[0]
df_viz_tmp['model'] = df_viz_tmp['model'].str.split('Qwen/').str[-1]

df_viz_tmp = df_viz_tmp.sort_values([
    'model_size',
    'adherent_and_correct'
])

df_viz_tmp = df_viz_tmp.astype({'n_few_shot_examples': str})

df_viz_tmp['encoding_scheme'] = df_viz_tmp['encoding_scheme'].map(lambda s: s.split("speaking_")[-1])

# Paper plots

In [ ]:
d_mapping = {
    "baseline": ["zero_shot", "identity"],
    "letter mutation": [
        "reverse_letters_in_each_word",
        "swap_even_odd_letters_in_each_word",
        "reverse_fibonacci_indices_in_each_word",
        "letter_to_word_with_dot",
        "dot_between_chars",
        "space_between_chars",
    ],
    "language deletion": ["remove_all_verbs", "remove_all_nouns"],
    "language translation": [
        "French","Chinese","Korean","Russian","Arabic","Adyghe",
        "Morse_code","Python","enterprise_Java",
    ],
    "algorithmic cipher": [
        "rot13_cipher","base64_cipher","base64_2x_cipher",
        "base64_3x_cipher","caesar_cipher","gzip_to_base64_encoded",
    ],
    "themed reasoning": [
        "paraphrase_naive",
        "pirate_speak",
        "leet_speak",
        "yoda_speak",
        "shakespearean_text",
    ],
    "extraneous content": [
        "insert_tweet",
        "python_snippet_comment",
        "croissant_news_article",
        "math_textbook_article",
        "five_emojis",
    ],
    "delete inf.": [
        "replace_math_content_with_black_box"
    ]
}

l_themed_encodings = [
    "remove_all_verbs",
    "paraphrase_naive",
    "pirate_speak",
    "leet_speak",
    "yoda_speak",
    "shakespearean_text",
    "insert_tweet",
    "python_snippet_comment",
    "croissant_news_article",
    "math_textbook_article",
    "five_emojis",
    "replace_math_content_with_black_box",
    "reverse_letters_in_each_word_no_math_expressions",
    "reverse_letters_in_each_word_only_math_expressions"
]

l_ignore_languages = [
    "Russian",
    "Chinese",
    # "Korean"
]

In [ ]:
import plotly.express as px
import plotly.colors as pc

def sample_colorscale(colorscale_name: str, n: int):
    """
    Sample N equally spaced hex colors from a Plotly continuous colorscale.

    Parameters
    ----------
    colorscale_name : str
        Name of a Plotly built-in continuous colorscale (e.g., "Viridis", "Blues").
    n : int
        Number of samples to return.

    Returns
    -------
    list of str
        List of hex color strings.
    """
    if n < 1:
        raise ValueError("n must be >= 1")
    colorscale = px.colors.get_colorscale(colorscale_name)
    # equally spaced points in [0,1]
    vals = [i/(n-1) if n > 1 else 0.5 for i in range(n)]
    return [pc.sample_colorscale(colorscale, v)[0] for v in vals]

l_gpt_gradient = sample_colorscale("Emrld", 4 + 1)
d_gpt_gradient = {
    model : color
    for model, color in zip(
        ['gpt-4.1-nano', 'gpt-4.1-mini', 'gpt-4.1', 'gpt-5-chat'],
        l_gpt_gradient[1:]
    )
}

l_claude_gradient = sample_colorscale("Magenta", 3 + 1)
d_claude_gradient = {
    model : color
    for model, color in zip(
        ['claude-3-opus-20240229', 'claude-3-5-sonnet-20241022', 'claude-sonnet-4-20250514'],
        l_claude_gradient[1:]
    )
}

l_qwen_gradient = sample_colorscale("Oryel", 3 + 1)
d_qwen_gradient = {
    model : color
    for model, color in zip(
        ['Qwen2.5-3B-Instruct', 'Qwen2.5-7B-Instruct', 'Qwen2.5-14B-Instruct'],
        l_qwen_gradient[1:]
    )
}

In [ ]:
l_star_encodings = [
    "reverse_letters_in_each_word",
    "rot13_cipher",
    "caesar_cipher", # TODO check
 'reverse_fibonacci_indices_in_each_word', # TODO check
 'swap_even_odd_letters_in_each_word', # TODO check
    "Morse_code"
]

In [ ]:
l_fixed_encoding_ordering = [
    # GPT
    'identity',
    'dot_between_chars',
    'Korean',
    'letter_to_word_with_dot',
    'rot13_cipher',
    # 'remove_all_verbs',
    'Morse_code',
    'base64_cipher',
    'reverse_letters_in_each_word',
    'reverse_fibonacci_indices_in_each_word',
'base64_2x_cipher',


    # placeholder for an empty space
    '',

    # Qwen/Claude
 'French',
 'space_between_chars',
 'Arabic',
 'Adyghe',
 'caesar_cipher',
 'swap_even_odd_letters_in_each_word',
    'remove_all_nouns',
 'Python',
    'enterprise_Java',
 'base64_3x_cipher',
 'gzip_to_base64_encoded',
 
]

# for i in range(len(l_fixed_encoding_ordering)):
#     if l_fixed_encoding_ordering[i] in l_star_encodings:
#         l_fixed_encoding_ordering[i] += "*"

In [ ]:
df_pgr[['encoding_scheme', 'model', 'PGR_pct', 'adherent_and_correct', 'generated_cot_is_correct', 'input_type']].to_parquet("/home/ubuntu/test.parquet")

In [ ]:
# --- Compute PGR (percent of identity performance) ---
df_pgr = df_viz_tmp.copy()

df_pgr = pd.concat([
    df_pgr,
    pd.DataFrame([{'encoding_scheme': '', 'model': model} for model in df_pgr['model'].unique()])
], ignore_index=True)

# Identity baseline per model
identity_baseline = (
    df_pgr[df_pgr["encoding_scheme"] == "identity"]
    .set_index("model")["adherent_and_correct"]
)

df_pgr["identity_value"] = np.maximum(df_pgr["model"].map(identity_baseline), 0.0001)

# Avoid divide-by-zero
# df_pgr = df_pgr[df_pgr["identity_value"] > 0].copy()

# Mean PGR as percentage
df_pgr["PGR_pct"] = (df_pgr["adherent_and_correct"] / df_pgr["identity_value"])

# CI deltas -> percentage deltas relative to identity baseline
# low is negative, high is positive
df_pgr["PGR_pct_hi_ci"]  = (df_pgr["adherent_and_correct_hi_ci"] / df_pgr["identity_value"])
df_pgr["PGR_pct_low_ci"] = (df_pgr["adherent_and_correct_low_ci"]        / df_pgr["identity_value"])

df_pgr.loc[df_pgr["encoding_scheme"] == "identity", ["PGR_pct_hi_ci", "PGR_pct_low_ci"]] = 0.0

# order by sonnet 4 hard coded
# df_sonnet4 = df_pgr[df_pgr['model'].str.contains('claude-3-5-sonnet')].sort_values('adherent_and_correct', ascending=False)
df_sonnet4 = df_pgr[df_pgr['model'].str.contains('14B')].sort_values('adherent_and_correct', ascending=False)
# df_sonnet4 = df_pgr[df_pgr['model'].str.contains('sonnet-4')].sort_values('adherent_and_correct', ascending=False)

d_encoding_scheme_to_idx = {}
for i in range(len(df_sonnet4)):
    d_encoding_scheme_to_idx[df_sonnet4['encoding_scheme'].iloc[i]] = i

# d_encoding_scheme_to_idx['identity'] = 0

for i, encoding in enumerate(l_fixed_encoding_ordering):
    d_encoding_scheme_to_idx[encoding] = i

df_pgr = df_pgr[~df_pgr['encoding_scheme'].isin(l_themed_encodings)]
df_pgr = df_pgr[~df_pgr['encoding_scheme'].isin(l_ignore_languages)]

# df_pgr['encoding_scheme'] = df_pgr['encoding_scheme'].map(lambda x: x + '*' if x in l_star_encodings else x)

df_pgr['sort_order'] = df_pgr['encoding_scheme'].map(d_encoding_scheme_to_idx)
df_pgr = df_pgr.sort_values(['sort_order', 'model_size'])

df_pgr['encoding_scheme'] = df_pgr['encoding_scheme'].map(lambda s: " ".join(s.split('_')))

df_pgr['Model Family'] = df_pgr["model"].map(lambda x: "Qwen2.5" if "Qwen2.5" in x else "GPT" if "gpt" in x else "Claude")

fig = px.line(df_pgr[df_pgr['encoding_scheme'] != 'zero shot'],
              x='encoding_scheme',
              y='PGR_pct',
              color='model',
              color_discrete_map={**d_claude_gradient, **d_gpt_gradient, **d_qwen_gradient},
             # barmode='group'
              markers=True,
              # facet_col="Model Family"
)

# s_encoding_scheme_order = df_pgr[df_pgr['model'].str.contains('14B') & (df_pgr['encoding_scheme'] != 'zero shot')]['encoding_scheme']
# s_encoding_scheme_order = df_pgr[df_pgr['model'].str.contains('sonnet-4') & (df_pgr['encoding_scheme'] != 'zero shot')]['encoding_scheme']
fig.update_layout(
    xaxis=dict(
        categoryorder="array",
        # categoryarray=s_encoding_scheme_order
        categoryarray=[' '.join(s.split('_')) for s in l_fixed_encoding_ordering]
    )
)

fig.update_layout(
    # title=dict(text=title, font=dict(size=22)),
    template="plotly_white",
    height=530,
    width=1800,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.08,
        xanchor="left",
        x=0.01,
        font=dict(size=22)
    ),
    margin=dict(t=0, r=0, b=0, l=0),
    font=dict(size=22),
    hovermode='x unified'
)

fig.update_xaxes(title="Cipher")
fig.update_yaxes(range=[0.0, 1.05], dtick=0.1, title="% of identity adherent & correct", title_font=dict(size=18), tickfont=dict(size=18))

# add the zero shot lines for 4.1 and 14b
l_zero_shot_models = ['gpt-4.1']
l_cols = [1]
l_pos = ["top right"]
l_model_names = ["GPT 4.1"]
# l_zero_shot_models = ['gpt-4.1-2025-04-14', "claude-sonnet-4-20250514"]
# l_cols = [1, 2]

for zero_shot_model, col_idx, pos, model_name in zip(l_zero_shot_models, l_cols, l_pos, l_model_names):
    zero_shot_perf = df_pgr[(df_pgr['model'] == zero_shot_model) & (df_pgr['encoding_scheme'] == 'zero shot')]['PGR_pct'].iloc[0]

    annotation_str = f"{model_name} direct answering perf."
    fig.add_hline(y=zero_shot_perf, line_dash='dash', annotation_text=annotation_str, col=col_idx, annotation_position=pos)
    # fig.add_hrect(
    #     # y0=0.0, y1=zero_shot_perf,
    #     y0=zero_shot_perf, y1=1.0,
    #     fillcolor="#D3D3D3", opacity=0.2,
    #     layer="below", line_width=0,
    #     annotation_text="Better than GPT 4.1 direct answering",
    #     annotation_font=dict(color="grey")
#     )

plotly.io.write_image(fig, 'sft_perf.pdf', format='pdf')

fig.show('png')


In [ ]:
import pandas as pd
import numpy as np

def df_to_latex_table(df_pgr,
                      caption="Model results",
                      label="tab:model-results",
                      percent_decimals=1,
                      float_decimals=2):
    from io import StringIO

    # 1) Build df_export
    df_export = df_pgr.copy()
    df_export['encoding_scheme'] = df_export['encoding_scheme'].map(
        lambda x: 'direct answering' if x == 'zero shot' else x
    )

    df_export = df_export[
        ['model', 'encoding_scheme', 'PGR_pct', 'generated_cot_is_correct',
         'generated_cot_adhered_encoding_style', 'backtranslation_bleu_scores',
         'Adherence Calculation Method']
    ].rename(columns={
        'encoding_scheme': 'encoding scheme',
        'PGR_pct': '\\% of identity adherent + coherent',
        'generated_cot_is_correct': 'MATH500 accuracy',
        'generated_cot_adhered_encoding_style': 'cipher adherence',
        'backtranslation_bleu_scores': 'translation BLEU',
    })

    # 2) Format numbers
    df_fmt = df_export.copy()

    col_pct = '\\% of identity adherent + coherent'
    if pd.api.types.is_numeric_dtype(df_fmt[col_pct]):
        s = df_fmt[col_pct].astype(float)
        if np.nanmax(s.values) <= 1.5:  # treat as fraction
            s = s * 100.0
        df_fmt[col_pct] = s.map(lambda x: (f"{x:.{percent_decimals}f}\\%" if pd.notna(x) else ""))

    numeric_cols = [
        'MATH500 accuracy',
        'cipher adherence',
        'translation BLEU',
    ]
    for c in numeric_cols:
        if c in df_fmt.columns and pd.api.types.is_numeric_dtype(df_fmt[c]):
            df_fmt[c] = df_fmt[c].map(lambda x: (f"{x:.{float_decimals}f}" if pd.notna(x) else ""))

    # 3) Build LaTeX manually with tabularx
    buffer = StringIO()
    col_names = df_fmt.columns.tolist()

    # Choose alignments: first two + last = text (wrap), others numeric
    col_spec = " | ".join(["p{" + f"{0.75 / len(col_names):.2f}" + "\\linewidth}" for _ in range(len(col_names))])

    # Write header
    buffer.write("\\begin{longtable}[]")
    buffer.write("{" + col_spec + "}\n")

    # Column headers
    buffer.write(" & ".join(col_names) + " \\\\\n")
    buffer.write("\\hline\n")

    # Rows
    for _, row in df_fmt.iterrows():
        buffer.write(" & ".join(str(x) for x in row.tolist()) + " \\\\\n")

    buffer.write("\\caption{" + caption + "}\n")
    buffer.write("\\label{" + label + "}\n")
    buffer.write("\\end{longtable}\n")

    latex = buffer.getvalue()
    print(latex)
    return latex



df_to_latex_table(df_pgr[df_pgr['PGR_pct'].notna()], caption="TODO CAPTION", label="detailed_sft_results_table")

None

In [ ]:
fig = px.line(df_pgr[df_pgr['encoding_scheme'] != 'zero shot'],
              x='encoding_scheme',
              y='backtranslation_bleu_scores',
              color='model',
              color_discrete_map={**d_claude_gradient, **d_gpt_gradient, **d_qwen_gradient},
             # barmode='group'
              markers=True,
              # facet_col="Model Family"
)

# needed for sorting by big model adherent+correct
fig.update_layout(
    xaxis=dict(
        categoryorder="array",
        # categoryarray=s_encoding_scheme_order
        categoryarray=[' '.join(s.split('_')) for s in l_fixed_encoding_ordering]
    )
)

fig.update_layout(
    # title=dict(text=title, font=dict(size=22)),
    template="plotly_white",
    height=500,
    width=1800,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.08,
        xanchor="left",
        x=0.01,
        font=dict(size=22)
    ),
    margin=dict(t=0, r=0, b=0, l=0),
    font=dict(size=22),
    hovermode='x unified'
)

fig.update_xaxes(title="Cipher")
fig.update_yaxes(range=[0, 101], dtick=20, title="BLEU score")

fig.add_hrect(
    y0=50, y1=100,
    fillcolor="#D3D3D3", opacity=0.25,
    layer="below", line_width=0,
    annotation_text="Fluent decoding",
    annotation_font=dict(color="grey")
)

# fig.update_layout(width=600, height=600, title=None)
fig.update_xaxes(tickangle=45)
fig.update_layout(height=600)

plotly.io.write_image(fig, 'sft_translation_perf.pdf', format='pdf')

fig.show('png')


In [ ]:
df_zero_shot_decode = pd.read_parquet("zero_shot_decode_data.parquet")

df_zero_shot_decode['model'] = df_zero_shot_decode['model'].str.split('-2025').str[0]

df_zero_shot_decode[['encoding_scheme', 'model']].drop_duplicates()

In [ ]:
# import numpy as np
# import pandas as pd
# import plotly.express as px
# import plotly.graph_objects as go
# from scipy.optimize import curve_fit

# import numpy as np
# import pandas as pd
# import plotly.graph_objects as go
# from plotly.subplots import make_subplots
# from scipy.optimize import curve_fit

# def styled_logistic_scatter_faceted(
#     df,
#     facet_col="model",
#     x_col="backtranslation_gt_logprobs",
#     y_col="adherent_and_correct",
#     title="Adherence vs. Backtranslation Log-Prob",
#     x_axis_title="Backtranslation log-prob",
#     y_axis_title="Adherent & correct",
#     font_family="Inter, Arial",
#     marker_size=7,
#     opacity=0.6,
#     legend_font_size=10,
#     x_tick_font_size=10,
#     line_width=3,
#     r2_digits=3,               # <— precision for R² display
#     keep_subplot_title=True,
#     max_pow_override=None
# ):
#     def logistic(x, L, k, x0):
#         return L / (1 + np.exp(-k * (x - x0)))

#     # overall y-scale to decide percent formatting
#     y_all = df[y_col].to_numpy(dtype=float)
#     is_percent = (np.nanmin(y_all) >= 0) and (np.nanmax(y_all) <= 1.0)

#     # facet ordering
#     if pd.api.types.is_categorical_dtype(df[facet_col]):
#         facet_values = [c for c in df[facet_col].cat.categories if (df[df[facet_col]==c].shape[0] > 0)]
#     else:
#         d_model_to_size = dict(zip(df['model'], df['model_size']))
#         facet_values = sorted(df[facet_col].dropna().unique(), key=lambda x: d_model_to_size.get(x, x))

#     print(facet_values)

#     n_cols = max(1, len(facet_values))
#     fig = make_subplots(
#         rows=1, cols=n_cols, shared_yaxes=True, horizontal_spacing=0.06,
#         subplot_titles=[f"{facet_col} = {v}" if keep_subplot_title else None for v in facet_values],
#     )

#     x_positive = df[df[x_col] > 0][x_col]
#     min_x, max_x = x_positive.min(), x_positive.max()
#     # Create tick values at decade intervals
#     min_pow = int(np.floor(np.log2(min_x)))
#     max_pow = int(np.ceil(np.log2(max_x)))
#     tick_vals = [2 ** p for p in range(min_pow, max_pow + 1)]
#     if max_pow_override:
#         if tick_vals[-1] > max_pow_override:
#             tick_vals[-1] = max_pow_override
#     tick_text = [f"{v:g}" for v in tick_vals]

#     for i, val in enumerate(facet_values, start=1):
#         sub = df[df[facet_col] == val]
#         x = sub[x_col].to_numpy(dtype=float)
#         y = sub[y_col].to_numpy(dtype=float)

#         # points
#         fig.add_trace(
#             go.Scatter(
#                 x=x, y=y, mode="markers+text", name=f"Points ({val})",
#                 marker=dict(size=marker_size, line=dict(width=0)),
#                 opacity=opacity, showlegend=False,
#                 text=sub["encoding_scheme"] + "-" + sub["model"],
#                 hovertemplate="<b>%{text}</b><extra></extra>",  # <-- Just encoding scheme
#             ),
#             row=1, col=i
#         )

#         # choose regressor space
#         # linear_on_log_x = True
#         # if linear_on_log_x:
#         #     valid = (x > 0) & np.isfinite(x) & np.isfinite(y)
#         #     X = np.log2(x[valid])
#         #     Y = y[valid]
#         # else:
#         #     valid = np.isfinite(x) & np.isfinite(y)
#         #     X = x[valid]
#         #     Y = y[valid]

#         # if X.size >= 2 and np.nanstd(Y) > 0:
#         #     b1, b0 = np.polyfit(X, Y, 1)   # Y = b1*X + b0
#         #     # make a smooth line across current x-range
#         #     x_line = np.linspace(np.nanmin(x[valid]), np.nanmax(x[valid]), 300)
#         #     X_line = np.log2(x_line) if linear_on_log_x else x_line
#         #     y_line = b1 * X_line + b0

#         #     # R^2 on observed points (in same space used to fit)
#         #     y_hat = b1 * X + b0
#         #     ss_res = np.nansum((Y - y_hat) ** 2)
#         #     ss_tot = np.nansum((Y - np.nanmean(Y)) ** 2)
#         #     if ss_tot > 0 and np.isfinite(ss_res):
#         #         r2 = 1 - ss_res / ss_tot
#         #         r2_text = f"{r2:.{r2_digits}f}"

#         #     fig.add_trace(
#         #         go.Scatter(
#         #             x=x_line, y=y_line, mode="lines",
#         #             name=f"Linear fit ({val})",
#         #             # line=linear_line_kwargs,
#         #             showlegend=(i == 1),
#         #         ),
#         #         row=1, col=i
#         #     )
        
#         # logistic fit + R^2
#         r2_text = "—"
#         try:
#             if np.isfinite(x).sum() >= 3 and np.nanstd(y) > 0:
#                 p0 = [np.nanmax(y), 1.0, np.nanmedian(x)]
#                 params, _ = curve_fit(logistic, x, y, p0=p0, maxfev=10000)
#                 L, k, x0 = params

#                 x_fit = np.linspace(np.nanmin(x), max(np.nanmax(x), 98), 300)
#                 y_fit = logistic(x_fit, L, k, x0)

#                 # R^2 on observed x
#                 y_hat = logistic(x, L, k, x0)
#                 ss_res = np.nansum((y - y_hat) ** 2)
#                 ss_tot = np.nansum((y - np.nanmean(y)) ** 2)
#                 if ss_tot > 0 and np.isfinite(ss_res):
#                     r2 = 1 - ss_res / ss_tot
#                     r2_text = f"{r2:.{r2_digits}f}"

#                 # fig.add_trace(
#                 #     go.Scatter(
#                 #         x=x_fit, y=y_fit, mode="lines",
#                 #         name=f"Logistic fit ({val})",
#                 #         line=dict(width=1, color="black", dash="dash"),
#                 #         showlegend=(i == 1),
#                 #     ),
#                 #     row=1, col=i
#                 # )
#         except Exception as e:
#             print(e)
#             pass

#         # R^2 annotation (top-left of this subplot)
#         # fig.add_annotation(
#         #     x=0.1, y=0.05,
#         #     xref=f"x{i}" if i > 1 else "x",   # ✅ Use proper axis names
#         #     yref=f"y{i}" if i > 1 else "y",
#         #     xanchor="left",
#         #     yanchor="bottom",
#         #     text=f"R² = {r2_text}",
#         #     showarrow=False,
#         #     font=dict(size=12),
#         #     align="left",
#         #     bgcolor="rgba(255,255,255,0.6)",
#         #     bordercolor="rgba(0,0,0,0.2)",
#         #     borderwidth=1,
#         # )

#         # axes cosmetics
#         fig.update_xaxes(
#             title=x_axis_title if i == 1 else "",
#             tickfont=dict(size=x_tick_font_size),
#             zeroline=False,
#             # type="log",
#             # autorange="reversed",
#             # tickvals=tick_vals,
#             # ticktext=tick_text,
#             row=1, col=i,
#             # dtick=0.1
#         )
#         fig.update_yaxes(
#             tickformat=".0%" if is_percent else ",",
#             rangemode="tozero",
#             zeroline=False,
#             dtick=0.05,
#             row=1, col=i,
#             title=y_axis_title
#         )

#     fig.update_layout(
#         template="plotly_white",
#         title=dict(text=title, font=dict(size=18)),
#         height=600,
#         width=350 * n_cols if n_cols <= 3 else 300 * n_cols,
#         font=dict(family=font_family, size=15),
#         margin=dict(t=0, r=0, b=0, l=0),
#         legend=dict(orientation="h", yanchor="bottom", y=1.08, xanchor="left", x=0,
#                     font=dict(size=legend_font_size)),
#         showlegend=False
#     )
#     # global axis labels
#     # fig.add_annotation(text=x_axis_title, showarrow=False, xref="paper", yref="paper",
#     #                    x=0.5, y=-0.18, font=dict(size=14))
#     # fig.add_annotation(text=y_axis_title, showarrow=False, xref="paper", yref="paper",
#     #                    x=-0.06, y=0.5, textangle=-90, font=dict(size=14))

#     return fig


# # ---- Use it on your dataframe ----
# df_viz_tmp_plot = df_viz_tmp.copy()

# df_viz_tmp_plot['encoding_scheme'] = df_viz_tmp_plot['encoding_scheme'].str.split('speaking_').str[-1]
# df_viz_tmp_plot['encoding_scheme'] = df_viz_tmp_plot['encoding_scheme'].str.replace("_", " ")

# df_zero_shot_decode['one_step_down_model'] = df_zero_shot_decode['model'].map({
#     'gpt-4.1-nano': 'gpt-4.1-mini',
#     'gpt-4.1-mini': 'gpt-4.1'
# })

# df_viz_tmp_plot = df_viz_tmp_plot.merge(df_zero_shot_decode, left_on=['model', 'encoding_scheme'], right_on=['one_step_down_model', 'encoding_scheme'], how='left', suffixes=('', '_zero_shot'))

# df_viz_tmp_plot["placeholder_col"] = "1"

# identity_baseline = (
#     df_viz_tmp_plot[df_viz_tmp_plot["encoding_scheme"] == "identity"]
#     .set_index("model")["adherent_and_correct"]
# )

# df_viz_tmp_plot["identity_value"] = np.maximum(df_viz_tmp_plot["model"].map(identity_baseline), 0.0001)

# # Avoid divide-by-zero
# # df_pgr = df_pgr[df_pgr["identity_value"] > 0].copy()

# # Mean PGR as percentage
# df_viz_tmp_plot["PGR_pct"] = (df_viz_tmp_plot["adherent_and_correct"] / df_viz_tmp_plot["identity_value"])

# df_viz_tmp_plot = df_viz_tmp_plot[~df_viz_tmp_plot['encoding_scheme'].isin(['identity', 'zero shot'])]

# # df_viz_tmp_plot = df_viz_tmp_plot[~df_viz_tmp_plot['model'].str.contains('mini')]
# # df_viz_tmp_plot = df_viz_tmp_plot[~df_viz_tmp_plot['model'].str.contains('nano')]


# fig = styled_logistic_scatter_faceted(
#     # df_viz_tmp_plot[df_viz_tmp_plot['backtranslation_bleu_scores'].notna() & ~df_viz_tmp_plot['model'].str.contains('gpt')],
#     #[df_viz_tmp_plot['adherent_and_correct'] > 0.01],
#     df_viz_tmp_plot,
#     # df_viz_tmp_plot[(df_viz_tmp_plot['adherent_and_correct'] >= df_viz_tmp_plot['model'].map(d_zero_shot_baseline))],
#     # x_col="backtranslation_gt_logprobs",
#     x_col="backtranslation_bleu_scores_zero_shot",
#     # y_col="adherent_and_correct",
#     y_col="PGR_pct",
#     # y_col='cot_gt_logprobs',
#     # title="% of responses adherent & correct vs. encoded -> English translation BLEU",
#     title=None,
#     x_axis_title="BLEU score",
#     # x_axis_title="Encoded text to English text log loss",
#     y_axis_title="% of identity adherent & correct",
#     # facet_col="model",
#     facet_col="placeholder_col",
#     keep_subplot_title=False,
#     max_pow_override=100,
#     x_tick_font_size=15
# )


# fig.update_layout(width=600, height=500, title=None)
# # fig.update_layout(width=1600, height=1200, title=None)
# # --- shaded region instead of vline ---
# x_cut = 60
# x_right = float(df_viz_tmp_plot["backtranslation_bleu_scores"].max()) + 5
# # x_cut = 0.2
# # x_right = 0.02


# # fig.add_vrect(
# #     x0=x_cut, x1=x_right,           # shade from cutoff to the right edge of data
# #     fillcolor="#D3D3D3", opacity=0.25,
# #     line_width=0,
# #     layer="below",
# #     row=1, col=1                    # adjust if you facet into multiple columns
# # )

# # # label for the shaded region
# # fig.add_annotation(
# #     x=0.99,
# #     # x=0.85,
# #     xref="paper",           # data coordinates
# #     y=0.99, yref="paper",
# #     text="Fluent<br>decoding",
# #     showarrow=False,
# #     xanchor="right",
# #     align="right",
# #     yanchor="top",
# #     bgcolor="rgba(255,255,255,0.0)",
# #     bordercolor="rgba(0,0,0,0.0)",
# #     borderwidth=0,
# #     font=dict(size=14)
# # )
# fig.update_traces(textposition="bottom center", opacity=0.8)
# fig.update_layout(font=dict(size=15))

# fig.update_yaxes(range=[-0.04, 1.0])
# fig.update_xaxes(range=[0, 100])

# fig.show()
# # fig.show()


In [ ]:
duckdb.query("""
SELECT MIN(PGR_pct), MAX(PGR_pct), MIN(generated_cot_is_correct), MAX(generated_cot_is_correct) FROM df_pgr
WHERE -- model LIKE '%gpt-4.1-2025%' AND
encoding_scheme NOT LIKE '%identity%'
AND backtranslation_bleu_scores > 95
-- AND generated_cot_is_correct > 0.05
""").to_df().T

In [ ]:
duckdb.query("""
SELECT * FROM df_pgr
WHERE 
model LIKE '%gpt-4.1%' AND
encoding_scheme LIKE '%fibonacci%'
""").to_df().T

In [ ]:
df_viz_tmp[(df_viz_tmp['encoding_scheme'] == 'base64_cipher') & (df_viz_tmp['model'].str.contains('4.1'))]

In [ ]:
df = pd.read_parquet("/home/ubuntu/sky_workdir/encoding-schemes/output/7b88a15cc7e51bd9ce792e47c494d2530577f989/data/joined_output.parquet")

df.head()

In [ ]:
idx = 194

In [ ]:
print(df['reference_problem'].iloc[idx])

In [ ]:
print(df['translated_solution'].iloc[idx])

In [ ]:
print(df['reference_solution'].iloc[idx])

In [ ]:
print(df['generated_backtranslations'].iloc[idx][0])

In [ ]:
df['backtranslation_bleu_scores'].iloc[idx][0]

In [ ]:
print(df['generated_cots'].iloc[idx][0])

In [ ]:
df['generated_cot_is_correct'].iloc[idx][0]

In [ ]:
from encoding_schemes.ciphers import inverse_base64_cipher

In [ ]:
# import codecs

# print(codecs.decode(df['generated_cots'].iloc[idx][2], 'rot13'))

import base64

print(inverse_base64_cipher(df['generated_cots'].iloc[idx][0]))